Importations de base

In [1]:
import json
import logging
import os
import pickle
import numpy as np
import pandas as pd
import joblib

In [2]:
# pip install azureml-core
# pip install azureml

In [3]:
# pip install azureml-automl-core

In [6]:
import azureml.automl.core

ModuleNotFoundError: No module named 'azureml.automl'

In [5]:
from azureml.automl.core.shared import logging_utilities, log_server
from azureml.telemetry import INSTRUMENTATION_KEY

ModuleNotFoundError: No module named 'azureml.automl'

In [ ]:
from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType
from inference_schema.parameter_types.standard_py_parameter_type import StandardPythonParameterType

In [ ]:
# Créer un échantillon de données pour le paramètre 'data'
data_sample = PandasParameterType(pd.DataFrame({"Manufacturer": pd.Series(["example_value"], dtype="object"), "Model": pd.Series(["example_value"], dtype="object"), "Prod. year": pd.Series(["2000-1-1"], dtype="datetime64[ns]"), "Category": pd.Series(["example_value"], dtype="object"), "Leather interior": pd.Series([False], dtype="bool"), "Fuel type": pd.Series(["example_value"], dtype="object"), "Engine volume": pd.Series(
    [0.0], dtype="float64"), "Mileage": pd.Series([0], dtype="int64"), "Cylinders": pd.Series([0.0], dtype="float64"), "Gear box type": pd.Series(["example_value"], dtype="object"), "Drive wheels": pd.Series(["example_value"], dtype="object"), "Doors": pd.Series([0], dtype="int64"), "Wheel": pd.Series([False], dtype="bool"), "Color": pd.Series(["example_value"], dtype="object"), "Airbags": pd.Series([0], dtype="int64"), "Turbo": pd.Series([False], dtype="bool")}))
# Créer un échantillon d'entrée pour le paramètre 'input'
input_sample = StandardPythonParameterType({'data': data_sample})

In [ ]:
# Créer un échantillon de résultats pour le paramètre 'Results'
result_sample = NumpyParameterType(np.array([0]))
# Créer un échantillon de sortie pour le paramètre 'output'
output_sample = StandardPythonParameterType({'Results': result_sample})
# Créer un échantillon de paramètres globaux pour le paramètre 'sample_global_parameters'
sample_global_parameters = StandardPythonParameterType(1.0)

In [ ]:
try:
    # Activer la télémétrie avec la clé d'instrumentation spécifiée
    log_server.enable_telemetry(INSTRUMENTATION_KEY)
    # Définir le niveau de verbosité sur 'INFO'
    log_server.set_verbosity('INFO')
    # Obtenir le logger pour le script de scoring v2
    logger = logging.getLogger('azureml.automl.core.scoring_script_v2')
except:
    # Ignorer les exceptions et continuer
    pass

In [ ]:
def init():
    global model
    # This name is model.id of model that we want to deploy deserialize the model file back
    # into a sklearn model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
    # Normaliser le chemin du modèle
    path = os.path.normpath(model_path)
    # Fractionner le chemin en segments
    path_split = path.split(os.sep)
    # Mettre à jour les dimensions personnalisées pour les journaux
    log_server.update_custom_dimensions(
        {'model_name': path_split[-3], 'model_version': path_split[-2]})
    try:
        # Charger le modèle à partir du chemin spécifié
        logger.info("Loading model from path.")
        model = joblib.load(model_path)
        logger.info("Loading successful.")
    except Exception as e:
        # Journaliser l'erreur et la remonter
        logging_utilities.log_traceback(e, logger)
        raise

In [ ]:
@input_schema('Inputs', input_sample)
@input_schema('GlobalParameters', sample_global_parameters, convert_to_provided_type=False)
@output_schema(output_sample)
def run(Inputs, GlobalParameters=1.0):
# Récupérer les données d'entrée
    data = Inputs['data']
    # Effectuer la prédiction en utilisant le modèle chargé
    result = model.predict(data)
    # Convertir le résultat en liste et le renvoyer dans le format spécifié
    return {'Results':result.tolist()}